In [1]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# 데이터 로드
data = pd.read_csv("IMDB Dataset.csv")

# DistilBERT 토크나이저
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# 토크나이징
input_texts = data["review"].tolist()
encoded_texts = tokenizer(input_texts, padding=True, truncation=True, return_tensors="tf")

# 레이블 처리
labels = np.array([1 if sentiment == "positive" else 0 for sentiment in data["sentiment"]])
labels = tf.constant(labels, dtype=tf.int64)

# 데이터셋 분할
train_inputs, test_inputs, train_labels, test_labels = train_test_split(encoded_texts, labels, test_size=0.2, random_state=42)

# DistilBERT 모델 생성
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

# 모델 학습
model.fit(train_inputs, train_labels, epochs=2, batch_size=16)

# 모델 평가
print("Evaluating the model...")
results = model.evaluate(test_inputs, test_labels)
print("Test loss, accuracy:", results)

# 예측용 파이프라인 생성
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, framework="tf")

# 예측 예시
example_text = "The movie was fantastic! I really enjoyed it."
predicted = pipeline(example_text)
print("Prediction:", predicted)

FileNotFoundError: [Errno 2] No such file or directory: 'IMDB Dataset.csv'